In [ ]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [ ]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo64'
DB_PASSWORD = 'grupo64'
DB_NAME = 'grupo64e2'

In [11]:
# Ruta del archivo CSV
archivo_csv = '../datos e2/platos.csv'

# Configuración de la conexión
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()

    # Dropear la tabla si ya existe
    drop_table_query = "DROP TABLE IF EXISTS platos;"
    cursor.execute(drop_table_query)
    print("Tabla platos eliminada si existía")

    # Crear la tabla (ajusta esto según tu esquema de datos)
    crear_tabla_query = """
    CREATE TABLE IF NOT EXISTS platos (
        id INT NOT NULL,
        nombre varchar   NOT NULL,
        descripcion varchar NOT NULL,
        disponibilidad bool   NOT NULL,
        estilo varchar   NOT NULL,
        restriccion varchar  NOT NULL,
        ingredientes varchar NOT NULL,
        porciones INT   NOT NULL,
        precio INT    NOT NULL,
        tiempo INT NOT NULL,
        restaurant varchar NOT NULL,
        repartomin INT NOT NULL,
        vigente bool   NOT NULL,
        CONSTRAINT pk_plato PRIMARY KEY (
            id))
    """
    cursor.execute(crear_tabla_query)
    conexion.commit()

    # Preparar el query de inserción
    insercion_query = """
    INSERT INTO platos (id, nombre, descripcion, disponibilidad, estilo, restriccion, ingredientes, porciones, precio, tiempo, restaurant, repartomin, vigente)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """

    # Leer y procesar el archivo CSV
    with open(archivo_csv, 'r') as file:
        lista_plato = []
        plato_actual = ""
        next(file)

        for linea in file:
            plato_actual += linea.strip()
            valores = plato_actual.split(';')
            if len(valores) == 13:
                lista_plato.append(valores)
                plato_actual = ""
            elif len(valores) > 13:
                print(f"Línea malformada: {plato_actual}")
                plato_actual = ""

        for plato in lista_plato:
            try:
                cursor.execute(insercion_query, (int(plato[0]), plato[1], plato[2], plato[3], plato[4], plato[5], plato[6], int(plato[7]), int(plato[8]), int(plato[9]), plato[10], int(plato[11]), plato[12]))
            except Error as e:
                    print(f"Error al insertar la línea {linea}: {e}")
                    conexion.rollback()
                    continue

    # Confirmar la inserción
    conexion.commit()

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

Tabla platos eliminada si existía


In [12]:
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()
    # Consulta para seleccionar todos los datos de la tabla 'clientes'
    seleccionar_query = "SELECT * FROM platos;"
    
    cursor.execute(seleccionar_query)
    # Obtener todos los registros
    registros = cursor.fetchall()

    # Imprimir los registros
    for registro in registros:
        print(registro)

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

(0, 'DCCHamburguer', 'Hamburguesa con palta, tomate, mayonesa, rica para pasar la hora de almuerzo', True, 'hamburguesa', 'celiacos', 'hamburguesa, palta, tomate, mayonesa', 1, 5000, 10, 'DCCRestaurant', 10000, True)
(1, 'Sushi', 'Una selecciÃ³n de rollitos de sushi tradicionales japoneses, servidos con wasabi y jengibre encurtido.', False, 'Internacional', 'pescetarianos', 'Arroz de sushi, algas nori, pescado crudo (como salmÃ³n, atÃºn o pez mantequilla), aguacate, pepino, cangrejo, vinagre de arroz, azÃºcar, sal', 3, 16000, 10, 'Burger Queen', 8000, False)
(2, 'Pizza Margarita', 'Una pizza clÃ¡sica italiana con salsa de tomate, mozzarella fresca y albahaca fresca.', False, 'fast food', 'celiacos', 'Masa de pizza, salsa de tomate, mozzarella fresca, albahaca fresca, aceite de oliva.', 2, 7000, 32, 'DCCRestaurant', 10000, True)
(3, 'Pizza Margarita', 'Una pizza clÃ¡sica italiana con salsa de tomate, mozzarella fresca y albahaca fresca.', True, 'fast food', 'celiacos', 'Masa de pizza, s